In [6]:
import warnings
warnings.filterwarnings('ignore')
import main
import ipywidgets as widgets
import tkinter as tk
from tkinter import filedialog
from IPython.core.display import clear_output, display, HTML
import qgrid
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import os
import pandas as pd
import dummy

In [7]:
search_engine_2 = main.SearchEngineElasticSearch(init_path=r'C:\Users\sergiojesus\Desktop\Document_Clustering\CV', index_name='cv')

In [8]:
style = {'description_width': 'initial'}
textWidget = widgets.Text(
                          placeholder='Search',
                          description='Search Term(s): ',
                          disabled=False,
                          style=style,
                          layout=widgets.Layout(width='1000px')
                         )

checkboxWidget = widgets.Checkbox(
                                  value=False, 
                                  description='Summary',
                                  disabled=False,
                                 )
results_output = widgets.Output()

def general_function(widget):
    if textWidget.value != '':
        results_output.clear_output()
        results = search_engine_2.query_database(textWidget.value, summary=checkboxWidget.value, max_size=10)
        with results_output:
            display(results[1])


textWidget.on_submit(general_function)
items = [textWidget, checkboxWidget]
boxLayout = widgets.Layout(flex_flow='line')
widgetsBox = widgets.HBox(children=items, layout=boxLayout)
resultsBox = widgets.HBox(children=[results_output])

In [10]:
display(HTML('<body><h1>CV Ingestion Framework</h1></body>'))
def upload_click(b):
    with open('temp.txt',"r") as pf:
        file_path = pf.read()
        filename = file_path.split('/')[-1:][0]
        
    with open(file_path,"r") as fp:
        corpus = fp.read()
    new_df = pd.DataFrame(index=[1], columns=["name", "content"], data={"name":filename, "content":corpus})

    col_names = new_df.columns
    for row_number in range(new_df.shape[0]):
        body = dict([(name, str(new_df.iloc[row_number][name])) for name in col_names])
        es.index(index = 'cv', doc_type= 'txt', body = body)
    print('>>> Upload to database done!')
    _bupload.button_style = 'success'
    _bupload.disabled = True
    
    
def select_click(b):
    #upload_dict = self._plotopt._create_dictionary()
    # upload_dict_fusion = self._plotopt._create_dictionary_fusion()
    #bulk_indexing(upload_dict, 1)
    # bulk_indexing_fusion(upload_dict_fusion, 1)
    _bupload.button_style = ''

    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(filetypes = (("pdf files","*.pdf"), ("txt files","*.txt"), ("all files","*.*")))
    filename = file_path.split('/')[-1:][0]
    extension = filename.split('.')[-1:][0]
    
    if extension == 'txt':
        print('>>> Selected file:',file_path.split('/')[-1:][0])
        _bupload.disabled = False
        _bupload.button_style = 'primary'
        with open('temp.txt', "w") as out:
            out.write(file_path)
        print('>>> Ready to Upload!')
            
    elif extension == 'pdf':
        print('>>> Selected file:',file_path.split('/')[-1:][0])
        print('>>> Please wait a few seconds for convertion...')
        txt = dummy.dummyTest(filename, file_path)
        with open(filename[:-3] + 'txt', "w") as fp:
            fp.write(txt)
        print('>>> PDF file', filename , 'converted to .txt!')
        
    else:
        _bupload.disabled = True
        print('>>> File must be in .txt')

es = Elasticsearch(['192.168.20.32:9200'])
_convert = widgets.Button(
                                   description='Converter',
                                   icon='picture-o'
                                   )      
_browse = widgets.Button(
                                   description='Browse',
                                   icon='picture-o'
                                   )        

_bupload = widgets.Button(
                                   description='Upload',
                                   icon='upload'
                                   )
_bupload.disabled = True

_convert.on_click(select_click)
_browse.on_click(select_click)
_bupload.on_click(upload_click)

box = widgets.HBox([_convert, _browse, _bupload])
display(box)
display(HTML("<style>div.input {display:none;}</style>"))  

>>> Selected file: CV_EmanuelOliveira_pt.pdf
>>> Please wait a few seconds for convertion...
>>> PDF file CV_EmanuelOliveira_pt.pdf converted to .txt!
>>> Selected file: CV_EmanuelOliveira_pt.txt
>>> Ready to Upload!


In [9]:
widgets.VBox(children=[widgetsBox, resultsBox])